## How to use the atlasofsmoothspaces package

First import

In [1]:
# import atlasofsmoothspaces as ass
import main as ass

Instantiate an instance with the desired smoothness types

In [24]:

CACHE_LENGTH = 5
NUMBER_OF_DERIVATIVES = 3
EXPONENTIAL_WEIGHTED_AVERAGE_ALPHA = 0.01

calibrationConfig = dict(
    smoothnessCacheLength = 3,
    autocalibrate = True,
    calibrationInterval = 4,
    calibrationStandardDevs = 2.5
)

sm = ass.Smoothness(
    ["GYRO_LEFT", "GYRO_RIGHT"], 
    cacheLengths = CACHE_LENGTH,
    derivativeDegrees = NUMBER_OF_DERIVATIVES,
    alphas = EXPONENTIAL_WEIGHTED_AVERAGE_ALPHA,
    **calibrationConfig)

Smoothness types can also be added and removed

First added

In [25]:
## EITHER existing pre-configured ones 
sm.initSmoothnessType("VEL_AVG",CACHE_LENGTH, NUMBER_OF_DERIVATIVES, EXPONENTIAL_WEIGHTED_AVERAGE_ALPHA, calibrationConfig=calibrationConfig)


## OR new customized ones
# First create the smoothness type
sm.addSmoothnessType(
    "TOTAL_ACCELERATION",
    lambda data : sm.smoothnessConversionTemplate(
        data=data, 
        channels=["CC{n}_1".format(n=m) for m in [19,20,21]], 
        aggregationType="euclidean"))

# and then initialized it with a cache and derivatives
sm.initSmoothnessType(
    smoothnessType="TOTAL_ACCELERATION", 
    cacheLength=4, 
    derivativeDegree=3,
    alpha=0.01,
    calibrationConfig=calibrationConfig)

# btw it can also be done in one call with  sm.addAndInitSmoothnessType(...)


check

In [26]:
sm.smoothnessTypes

['GYRO_LEFT', 'GYRO_RIGHT', 'VEL_AVG', 'TOTAL_ACCELERATION']

Or removed

In [27]:
sm.removeSmoothnessType("VEL_AVG")

In [28]:
sm.data.keys()

dict_keys(['GYRO_LEFT', 'GYRO_RIGHT', 'TOTAL_ACCELERATION'])

Channels can be added and removed, too

In [29]:
sm.addChannel("CC23_1", "gyro xz")
sm.addChannel("CC23_2", "gyro xzy")
sm.removeChannel("CC23_2")

check

In [30]:
sm.channels

{'CC16_1': 'gyro x left',
 'CC16_2': 'gyro x right',
 'CC17_1': 'gyro y left',
 'CC17_2': 'gyro y right',
 'CC18_1': 'gyro z left',
 'CC18_2': 'gyro z right',
 'CC19_1': 'accel x left',
 'CC19_2': 'accel x right',
 'CC20_1': 'accel y left',
 'CC20_2': 'accel y right',
 'CC21_1': 'accel z left',
 'CC21_2': 'accel z right',
 'CC22_1': 'vel left',
 'CC22_2': 'vel right',
 'CC23_1': 'gyro xz'}

you need to feed it with new data using the following format (best is to use all the channel data)

Make sure that all the smoothness types will be supplied with data. In the below example we would have to add the velocity channels to cover also the VEL_LEFT and VEL_RIGHT smoothness types.

You can also get the aggregated smoothness value as a return value  (optional)

In [51]:
import random
sm.addNewValues({
    'CC16_1': random.randint(0,ass.MIDI_MAX),
    'CC17_1': random.randint(0,ass.MIDI_MAX),
    'CC18_1': random.randint(0,ass.MIDI_MAX),
    'CC16_2': random.randint(0,ass.MIDI_MAX),
    'CC17_2': random.randint(0,ass.MIDI_MAX),
    'CC18_2': random.randint(0,ass.MIDI_MAX),
    'CC19_1': random.randint(0,ass.MIDI_MAX),
    'CC20_1': random.randint(0,ass.MIDI_MAX),
    'CC21_1': random.randint(0,ass.MIDI_MAX),
},
returnSmoothness=True)

{'GYRO_LEFT': {'raw': 2.2743578893015815e-05,
  'calibratedSignal': 49.98735038808885},
 'GYRO_RIGHT': {'raw': 1.118803613519452e-05,
  'calibratedSignal': 12.726443286680297},
 'TOTAL_ACCELERATION': {'raw': 0.0008603644330756963,
  'calibratedSignal': 126.99999563162378}}

execute the above code a few times. when the returnFlag is enabled and autocalibration also, then you'll realize that after a few executions the calibrated signal is also shown. It works quite well. It's within the range of 0 to 127. You can also disable the return flag in which case you won't get any feedback ...

... but you can at any time also query the smoothness of each smoothness type

In [47]:
sm.getSmoothnessMeasure()

{'GYRO_LEFT': {'raw': 3.862251861610559e-06},
 'GYRO_RIGHT': {'raw': 3.127720486684677e-06},
 'TOTAL_ACCELERATION': {'raw': 0.0003136696025079371}}

### Calibrate

If autocalibrate is switched on or was switched on by default. You can change the calibration by changing the sensitivity. Play around with it a bit. A higher sensitivity makes the values jump from 0 to 127. A lower sensitivity (i.e. higher insensitivity) makes the values revolve around similar values

In [93]:
sm.updateInsensitivityForAutocalibration(newInsensitivities=100) ## maybe try 100 ( very insensitive ) or 0.1 ( very sensitive ) 

In [94]:
sm.data["GYRO_LEFT"].smoothnessMeasures["overall"]

{'raw': 1.3860263361197518e-05,
 'calibrationCount': 56,
 'calibrationStandardDevs': 100,
 'calibration': {'XMIN': 1.6716356190804567e-05,
  'XMAX': 2.303046613329722e-05,
  'YMIN': 0,
  'YMAX': 127,
  'BETA': 0.02},
 'cache': array([2.48765777e-05, 1.69021562e-05, 1.38602634e-05]),
 'calibratedSignal': 126.99976361812817,
 'isCalibrated': True}

In [109]:
sm.addNewValues({
    'CC16_1': random.randint(0,ass.MIDI_MAX),
    'CC17_1': random.randint(0,ass.MIDI_MAX),
    'CC18_1': random.randint(0,ass.MIDI_MAX),
    'CC16_2': random.randint(0,ass.MIDI_MAX),
    'CC17_2': random.randint(0,ass.MIDI_MAX),
    'CC18_2': random.randint(0,ass.MIDI_MAX),
    'CC19_1': random.randint(0,ass.MIDI_MAX),
    'CC20_1': random.randint(0,ass.MIDI_MAX),
    'CC21_1': random.randint(0,ass.MIDI_MAX),
},
returnSmoothness=True)

{'GYRO_LEFT': {'raw': 2.295643152108795e-05,
  'calibratedSignal': 3.1743166317020055},
 'GYRO_RIGHT': {'raw': 4.284924201914617e-06,
  'calibratedSignal': 2.605092658812452},
 'TOTAL_ACCELERATION': {'raw': 0.0024215079977728342,
  'calibratedSignal': 2.765550967438672}}

If autocalibrate is switched off. Then before you manually calibrate, you first repeatedly feed it a few values (using the function above) so that it can find its equilibrium easier.

for instance in my experiments with random numbers the gyro left is always between 5*10**(-7) and 5*10**(-6)

In [111]:
## optional in case you have it switched on:
sm.stopAutocalibration()

## switch on calibration manually
sm.calibrateSmoothness(
    [
        "GYRO_LEFT",
        "GYRO_RIGHT",
        "TOTAL_ACCELERATION"
    ],
    [
        [2.5*10**(-6), 4*10**(-5)],
        [2.5*10**(-6), 4*10**(-5)],
        [2.5*10**(-6), 4*10**(-5)]
    ])

Now when you add numbers you get the calibrated result between 0 and 127

In [113]:

sm.addNewValues({
    'CC16_1': random.randint(0,ass.MIDI_MAX),
    'CC17_1': random.randint(0,ass.MIDI_MAX),
    'CC18_1': random.randint(0,ass.MIDI_MAX),
    'CC16_2': random.randint(0,ass.MIDI_MAX),
    'CC17_2': random.randint(0,ass.MIDI_MAX),
    'CC18_2': random.randint(0,ass.MIDI_MAX),
    'CC19_1': random.randint(0,ass.MIDI_MAX),
    'CC20_1': random.randint(0,ass.MIDI_MAX),
    'CC21_1': random.randint(0,ass.MIDI_MAX)
},
returnSmoothness=True)

{'GYRO_LEFT': {'raw': 2.2739483833127158e-05,
  'calibratedSignal': 88.38528894607555},
 'GYRO_RIGHT': {'raw': 4.254414236078145e-06,
  'calibratedSignal': 5.973035398971157},
 'TOTAL_ACCELERATION': {'raw': 0.002386607877044301,
  'calibratedSignal': 127.0}}